# Day13_0: 회귀 모델 (Regression Models) - 정답

---

In [ ]:
# 환경 설정
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import fetch_california_housing
from scipy.stats import randint, uniform

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

# 데이터 준비
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target

X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("데이터 준비 완료!")

---

## Q1. 선형 회귀 기본 ⭐

**문제**: LinearRegression 모델을 학습하고 R2 스코어를 계산하세요.

In [ ]:
# 정답 코드
# 1. 모델 생성 및 학습
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# 2. 예측
y_pred = lr_model.predict(X_test_scaled)

# 3. R2 스코어 계산
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2:.4f}")

In [ ]:
# 검증
assert 0.5 < r2 < 0.7, "R2 스코어가 예상 범위를 벗어났습니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. LinearRegression 인스턴스 생성
2. fit()으로 학습 (스케일링된 데이터 사용)
3. predict()로 예측
4. r2_score()로 평가

**핵심 개념**:
- 선형 회귀는 스케일링된 데이터에서 더 안정적
- R2는 0~1 사이 값으로, 1에 가까울수록 좋음

**대안**:
- 스케일링 없이도 가능하지만 계수 해석이 어려움

**흔한 실수**:
- 학습과 테스트에 다른 스케일러 적용
- fit_transform을 테스트에도 사용 (정보 누출)

**실무 팁**:
- 선형 회귀는 항상 기준 모델로 먼저 시도

---

## Q2. Ridge 정규화 ⭐

**문제**: alpha=10인 Ridge 모델을 학습하고 RMSE를 계산하세요.

In [ ]:
# 정답 코드
# 1. Ridge 모델 생성 (alpha=10)
ridge_model = Ridge(alpha=10)

# 2. 학습
ridge_model.fit(X_train_scaled, y_train)

# 3. 예측
y_pred_ridge = ridge_model.predict(X_test_scaled)

# 4. RMSE 계산
rmse = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
print(f"RMSE: {rmse:.4f}")

In [ ]:
# 검증
assert 0.7 < rmse < 0.8, "RMSE가 예상 범위를 벗어났습니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. Ridge(alpha=10)으로 정규화 강도 설정
2. RMSE = sqrt(MSE)로 계산

**핵심 개념**:
- alpha가 클수록 계수가 작아짐 (정규화 강함)
- RMSE는 원래 단위로 해석 가능 ($100,000)

**대안**:
- from sklearn.metrics import mean_squared_error + np.sqrt()
- 또는 sklearn 1.4+에서는 root_mean_squared_error 사용

**흔한 실수**:
- MSE와 RMSE 혼동
- sqrt 적용 누락

**실무 팁**:
- alpha 값은 교차 검증으로 결정

---

## Q3. Lasso 특성 선택 ⭐⭐

**문제**: alpha=0.1인 Lasso 모델을 학습하고, 0이 아닌 계수의 개수를 출력하세요.

In [ ]:
# 정답 코드
# 1. Lasso 모델 생성 (alpha=0.1)
lasso_model = Lasso(alpha=0.1)

# 2. 학습
lasso_model.fit(X_train_scaled, y_train)

# 3. 0이 아닌 계수 개수 계산
non_zero_count = np.sum(lasso_model.coef_ != 0)

print(f"전체 특성 수: {len(X.columns)}")
print(f"선택된 특성 수 (0이 아닌 계수): {non_zero_count}")

# 선택된 특성 확인
selected_features = X.columns[lasso_model.coef_ != 0]
print(f"선택된 특성: {list(selected_features)}")

In [ ]:
# 검증
assert non_zero_count <= len(X.columns), "계수 개수가 특성 수를 초과할 수 없습니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. Lasso의 L1 정규화는 계수를 0으로 만들 수 있음
2. np.sum(coef_ != 0)으로 0이 아닌 개수 계산

**핵심 개념**:
- Lasso는 자동 특성 선택 기능
- alpha가 클수록 더 많은 특성이 0이 됨

**대안**:
- np.count_nonzero(lasso_model.coef_)

**흔한 실수**:
- coef_.sum()으로 개수 계산 (값의 합계와 혼동)

**실무 팁**:
- 특성이 많을 때 Lasso로 핵심 변수만 추출

---

## Q4. Random Forest 학습 ⭐⭐

**문제**: n_estimators=50, max_depth=10인 Random Forest를 학습하고 성능을 평가하세요.

In [ ]:
# 정답 코드
# 1. Random Forest 모델 생성
rf_model = RandomForestRegressor(
    n_estimators=50,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

# 2. 학습 (스케일링 불필요)
rf_model.fit(X_train, y_train)

# 3. 예측
y_pred_rf = rf_model.predict(X_test)

# 4. 성능 평가
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

print(f"R2 Score: {r2_rf:.4f}")
print(f"RMSE: {rmse_rf:.4f}")

In [ ]:
# 검증
assert r2_rf > 0.7, "R2가 0.7보다 작습니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. RandomForestRegressor 생성
2. 원본 데이터로 학습 (스케일링 불필요)
3. R2와 RMSE 모두 계산

**핵심 개념**:
- n_estimators: 트리 개수 (많을수록 안정적)
- max_depth: 트리 깊이 (과적합 방지)
- n_jobs=-1: 모든 CPU 코어 사용

**대안**:
- n_jobs 생략 가능 (단일 코어)

**흔한 실수**:
- 트리 모델에 스케일링 적용 (불필요하지만 해도 됨)

**실무 팁**:
- random_state 고정으로 재현성 확보

---

## Q5. 특성 중요도 추출 ⭐⭐

**문제**: Random Forest의 특성 중요도를 DataFrame으로 만들고 상위 3개를 출력하세요.

In [ ]:
# 정답 코드
# 1. 특성 중요도 DataFrame 생성
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
})

# 2. 중요도 순으로 정렬
importance_df = importance_df.sort_values('Importance', ascending=False)

# 3. 상위 3개 출력
print("=== 특성 중요도 Top 3 ===")
print(importance_df.head(3))

In [ ]:
# 검증
assert importance_df.shape[0] == len(X.columns), "특성 수가 일치하지 않습니다"
assert importance_df['Importance'].sum() - 1.0 < 0.01, "중요도 합이 1이 아닙니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. feature_importances_ 속성으로 중요도 추출
2. DataFrame으로 변환
3. sort_values + head(3)

**핵심 개념**:
- 특성 중요도는 해당 특성의 불순도 감소 기여도
- 합계가 1이 되도록 정규화됨

**대안**:
- nlargest(3, 'Importance') 사용

**흔한 실수**:
- ascending=True로 정렬 (낮은 순)

**실무 팁**:
- 중요도 낮은 특성 제거로 모델 간소화

---

## Q6. XGBoost 학습 ⭐⭐⭐

**문제**: learning_rate=0.05, max_depth=8인 XGBoost 모델을 학습하고 성능을 평가하세요.

In [ ]:
# 정답 코드
# 1. XGBoost 모델 생성
xgb_model = xgb.XGBRegressor(
    learning_rate=0.05,
    max_depth=8,
    n_estimators=100,  # 기본값
    random_state=42,
    verbosity=0
)

# 2. 학습
xgb_model.fit(X_train, y_train)

# 3. 예측
y_pred_xgb = xgb_model.predict(X_test)

# 4. 성능 평가
r2_xgb = r2_score(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

print(f"R2 Score: {r2_xgb:.4f}")
print(f"RMSE: {rmse_xgb:.4f}")

In [ ]:
# 검증
assert r2_xgb > 0.8, "XGBoost R2가 0.8보다 작습니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. XGBRegressor 생성
2. learning_rate와 max_depth 설정
3. verbosity=0으로 출력 억제

**핵심 개념**:
- learning_rate: 각 트리의 기여도 (작을수록 안정적)
- max_depth: 개별 트리 깊이
- n_estimators와 learning_rate는 trade-off

**대안**:
- eta 대신 learning_rate 사용 (동일)

**흔한 실수**:
- learning_rate가 너무 높으면 과적합

**실무 팁**:
- learning_rate 낮추고 n_estimators 높이면 성능 향상

---

## Q7. LightGBM 학습 ⭐⭐⭐

**문제**: n_estimators=200, num_leaves=31인 LightGBM 모델을 학습하고 R2를 계산하세요.

In [ ]:
# 정답 코드
# 1. LightGBM 모델 생성
lgb_model = lgb.LGBMRegressor(
    n_estimators=200,
    num_leaves=31,
    random_state=42,
    verbose=-1
)

# 2. 학습
lgb_model.fit(X_train, y_train)

# 3. 예측
y_pred_lgb = lgb_model.predict(X_test)

# 4. R2 계산
r2_lgb = r2_score(y_test, y_pred_lgb)

print(f"R2 Score: {r2_lgb:.4f}")

In [ ]:
# 검증
assert r2_lgb > 0.8, "LightGBM R2가 0.8보다 작습니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. LGBMRegressor 생성
2. num_leaves는 LightGBM 특유의 파라미터
3. verbose=-1로 경고 억제

**핵심 개념**:
- num_leaves: 리프 노드 수 (2^max_depth보다 작게)
- LightGBM은 Leaf-wise 분할 (더 효율적)

**대안**:
- max_depth 대신 num_leaves로 복잡도 제어

**흔한 실수**:
- num_leaves가 너무 크면 과적합

**실무 팁**:
- 대용량 데이터에서 LightGBM이 XGBoost보다 빠름

---

## Q8. GridSearchCV 실습 ⭐⭐⭐⭐

**문제**: Ridge의 alpha를 [0.01, 0.1, 1, 10, 100] 중에서 GridSearchCV로 최적화하세요.

In [ ]:
# 정답 코드
# 1. 파라미터 그리드 정의
param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100]
}

# 2. 기본 모델
ridge_base = Ridge()

# 3. GridSearchCV 설정
grid_search = GridSearchCV(
    ridge_base,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

# 4. 학습
grid_search.fit(X_train_scaled, y_train)

# 5. 결과 출력
print(f"최적 alpha: {grid_search.best_params_['alpha']}")
print(f"최적 CV RMSE: {np.sqrt(-grid_search.best_score_):.4f}")

# 6. 전체 결과 확인
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results['RMSE'] = np.sqrt(-cv_results['mean_test_score'])
print("\n=== Alpha별 성능 ===")
print(cv_results[['param_alpha', 'RMSE']].to_string(index=False))

In [ ]:
# 검증
assert grid_search.best_params_['alpha'] in [0.01, 0.1, 1, 10, 100]
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. param_grid에 탐색할 값 정의
2. cv=5로 5-fold 교차 검증
3. scoring='neg_mean_squared_error' (음수 MSE)
4. best_params_, best_score_로 결과 확인

**핵심 개념**:
- GridSearchCV는 모든 조합 시도
- sklearn scoring은 "높을수록 좋음"이 기본 → MSE에 음수

**대안**:
- scoring='r2' 사용 가능

**흔한 실수**:
- neg_mean_squared_error에서 음수 처리 누락

**실무 팁**:
- return_train_score=True로 과적합 확인

---

## Q9. RandomizedSearchCV 실습 ⭐⭐⭐⭐

**문제**: Random Forest의 하이퍼파라미터를 RandomizedSearchCV로 최적화하세요.
- n_estimators: 50~200
- max_depth: 5~20
- n_iter=10

In [ ]:
# 정답 코드
# 1. 파라미터 분포 정의
param_dist = {
    'n_estimators': randint(50, 201),  # 50~200
    'max_depth': randint(5, 21)        # 5~20
}

# 2. 기본 모델
rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)

# 3. RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    rf_base,
    param_dist,
    n_iter=10,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

# 4. 학습
random_search.fit(X_train, y_train)

# 5. 결과 출력
print("최적 파라미터:")
print(f"  n_estimators: {random_search.best_params_['n_estimators']}")
print(f"  max_depth: {random_search.best_params_['max_depth']}")
print(f"\n최적 CV RMSE: {np.sqrt(-random_search.best_score_):.4f}")

# 6. 최적 모델로 테스트
best_rf = random_search.best_estimator_
y_pred_best = best_rf.predict(X_test)
print(f"테스트 R2: {r2_score(y_test, y_pred_best):.4f}")

In [ ]:
# 검증
assert 50 <= random_search.best_params_['n_estimators'] <= 200
assert 5 <= random_search.best_params_['max_depth'] <= 20
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. scipy.stats.randint로 정수 분포 정의
2. n_iter=10으로 10번만 탐색
3. random_state 고정으로 재현성 확보

**핵심 개념**:
- RandomizedSearchCV는 무작위 샘플링
- GridSearchCV보다 효율적 (대규모 탐색)
- n_iter로 탐색 횟수 제어

**대안**:
- uniform()으로 실수 분포 정의
- loguniform()으로 로그 스케일

**흔한 실수**:
- randint(50, 200) → 50~199 (상한 미포함)

**실무 팁**:
- n_iter는 시간과 정확도의 trade-off

---

## Q10. 모델 비교 종합 ⭐⭐⭐⭐⭐

**문제**: 3개 이상의 모델을 학습하고 RMSE와 R2를 비교하는 DataFrame을 만든 후, Plotly로 시각화하세요.

In [ ]:
# 정답 코드
# 1. 여러 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, max_depth=6, verbosity=0, random_state=42),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, verbose=-1, random_state=42),
    'CatBoost': cb.CatBoostRegressor(n_estimators=100, max_depth=6, verbose=0, random_state=42)
}

# 2. 결과 저장 리스트
results = []

# 3. 각 모델 학습 및 평가
for name, model in models.items():
    # 선형 모델은 스케일링된 데이터 사용
    if name in ['Linear Regression', 'Ridge']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # 평가 지표 계산
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'RMSE': rmse,
        'R2': r2
    })

# 4. DataFrame 생성
results_df = pd.DataFrame(results).sort_values('RMSE')
print("=== 모델 비교 결과 ===")
print(results_df.to_string(index=False))

In [ ]:
# 5. Plotly 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['RMSE (낮을수록 좋음)', 'R2 Score (높을수록 좋음)'])

# RMSE 바 차트
fig.add_trace(
    go.Bar(
        x=results_df['Model'], 
        y=results_df['RMSE'], 
        name='RMSE',
        marker_color='indianred',
        text=results_df['RMSE'].round(4),
        textposition='outside'
    ),
    row=1, col=1
)

# R2 바 차트
fig.add_trace(
    go.Bar(
        x=results_df['Model'], 
        y=results_df['R2'], 
        name='R2',
        marker_color='seagreen',
        text=results_df['R2'].round(4),
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title='회귀 모델 성능 비교',
    height=500,
    showlegend=False
)
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
# 검증
assert len(results_df) >= 3, "최소 3개 모델이 필요합니다"
assert 'RMSE' in results_df.columns and 'R2' in results_df.columns
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 딕셔너리로 여러 모델 정의
2. for 루프로 학습/평가 반복
3. 선형/트리 모델 스케일링 분기 처리
4. make_subplots로 두 지표 동시 시각화

**핵심 개념**:
- 모델 비교는 동일 데이터셋에서 수행
- RMSE와 R2 모두 확인 (상호보완)
- 시각화로 직관적 비교

**대안**:
- px.bar()로 간단히 시각화
- cross_val_score로 더 robust한 비교

**흔한 실수**:
- 스케일링 필요 모델에 원본 데이터 사용
- 동일 평가 데이터 미사용

**실무 팁**:
- 항상 baseline(Linear Regression) 포함
- 교차 검증으로 분산도 확인

---

## Q11. CatBoost 기본 학습 ⭐⭐⭐

**문제**: CatBoostRegressor를 사용하여 회귀 모델을 학습하세요.
- n_estimators=100, max_depth=6, learning_rate=0.1
- R2 스코어와 RMSE를 출력하세요.

In [ ]:
# 정답 코드
# 1. CatBoost 모델 생성
cat_model = cb.CatBoostRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    verbose=0  # 출력 숨김
)

# 2. 학습
cat_model.fit(X_train, y_train)

# 3. 예측
y_pred_cat = cat_model.predict(X_test)

# 4. 성능 평가
r2_cat = r2_score(y_test, y_pred_cat)
rmse_cat = np.sqrt(mean_squared_error(y_test, y_pred_cat))

print(f"R2 Score: {r2_cat:.4f}")
print(f"RMSE: {rmse_cat:.4f}")

In [ ]:
# 검증
assert r2_cat > 0.8, "CatBoost R2가 0.8보다 작습니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. CatBoostRegressor 생성
2. verbose=0으로 학습 로그 숨김
3. XGBoost/LightGBM과 동일한 사용법

**핵심 개념**:
- CatBoost는 Yandex에서 개발한 Gradient Boosting 라이브러리
- **범주형 특성 자동 처리**가 핵심 장점
- Ordered Target Statistics로 타겟 누수 방지
- 대칭 트리(Oblivious Trees) 사용으로 예측 속도 빠름

**대안**:
- iterations 대신 n_estimators 사용 (동일)
- depth 대신 max_depth 사용 (동일)

**흔한 실수**:
- verbose 설정 안 하면 학습 로그가 많이 출력됨
- silent=True (deprecated) 대신 verbose=0 사용

**실무 팁**:
- 범주형 특성이 많은 데이터셋에서 CatBoost가 특히 강점
- cat_features 파라미터로 범주형 컬럼 인덱스 지정 가능

---

## Q12. 부스팅 3대장 비교 ⭐⭐⭐⭐

**문제**: XGBoost, LightGBM, CatBoost 세 가지 모델을 모두 학습하고, 성능(RMSE, R2)을 비교하는 DataFrame을 만드세요.
- 동일한 하이퍼파라미터 사용: n_estimators=100, max_depth=6, learning_rate=0.1
- 어떤 모델이 가장 좋은 성능을 보이나요?

In [ ]:
# 정답 코드
# 1. 부스팅 3대장 모델 정의 (동일 하이퍼파라미터)
boosting_models = {
    'XGBoost': xgb.XGBRegressor(
        n_estimators=100, max_depth=6, learning_rate=0.1,
        random_state=42, verbosity=0
    ),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=100, max_depth=6, learning_rate=0.1,
        random_state=42, verbose=-1
    ),
    'CatBoost': cb.CatBoostRegressor(
        n_estimators=100, max_depth=6, learning_rate=0.1,
        random_state=42, verbose=0
    )
}

# 2. 결과 저장
boosting_results = []

# 3. 각 모델 학습 및 평가
for name, model in boosting_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    boosting_results.append({
        'Model': name,
        'RMSE': rmse,
        'R2': r2
    })

# 4. DataFrame 생성 및 정렬
boosting_df = pd.DataFrame(boosting_results).sort_values('RMSE')
print("=== 부스팅 3대장 성능 비교 ===")
print(boosting_df.to_string(index=False))

# 5. 최고 성능 모델
best_model = boosting_df.iloc[0]['Model']
print(f"\n가장 좋은 성능: {best_model}")

In [ ]:
# 검증
assert len(boosting_df) == 3, "3개 모델이 모두 포함되어야 합니다"
assert all(boosting_df['R2'] > 0.8), "모든 모델의 R2가 0.8 이상이어야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 세 모델에 동일한 하이퍼파라미터 적용
2. for 루프로 학습/평가 반복
3. RMSE 기준 정렬로 최고 성능 확인

**핵심 개념**:
- **XGBoost**: Level-wise 트리 성장, 가장 범용적
- **LightGBM**: Leaf-wise 트리 성장, 속도가 가장 빠름
- **CatBoost**: 대칭 트리, 범주형 자동 처리

**부스팅 3대장 비교표**:

| 특성 | XGBoost | LightGBM | CatBoost |
|------|---------|----------|----------|
| 개발사 | DMLC | Microsoft | Yandex |
| 속도 | 빠름 | 가장 빠름 | 보통 |
| 범주형 처리 | 수동 인코딩 | 수동 인코딩 | 자동 |
| 추천 상황 | 일반적 | 대용량 | 범주형 많을 때 |

**흔한 실수**:
- 세 모델에 다른 하이퍼파라미터 사용 (공정한 비교 불가)
- verbose 설정 누락으로 로그 과다 출력

**실무 팁**:
- 데이터셋 특성에 따라 최적 모델이 다름
- 범주형 특성 많으면 CatBoost 우선 시도
- 대용량 데이터면 LightGBM 추천
- 처음 시작한다면 XGBoost가 안전한 선택

---

## 📊 학습 정리

### Part 1: 선형 모델 핵심 요약

| 모델 | 특징 | 언제 사용? |
|------|------|------------|
| Linear Regression | 가장 기본, 해석 용이 | 기준 모델, 선형 관계 |
| Ridge (L2) | 계수 축소 | 다중공선성, 과적합 방지 |
| Lasso (L1) | 특성 선택 | 핵심 변수 추출 |
| ElasticNet | L1+L2 조합 | Ridge와 Lasso 장점 결합 |

### Part 2: 비선형 모델 & 튜닝 요약

| 모델 | 특징 | 핵심 파라미터 |
|------|------|---------------|
| Decision Tree | 비선형 학습 | max_depth, min_samples_split |
| Random Forest | 앙상블, 안정적 | n_estimators, max_depth |
| XGBoost | 고성능, Kaggle 인기 | learning_rate, max_depth |
| LightGBM | 빠른 속도, 대용량 | num_leaves, learning_rate |
| CatBoost | 범주형 자동 처리 | cat_features, learning_rate |

### 💡 부스팅 3대장 선택 가이드

| 상황 | 추천 모델 |
|------|----------|
| 범주형 특성이 많을 때 | **CatBoost** |
| 대용량 데이터 | **LightGBM** |
| 안정적인 성능 필요 | **XGBoost** |
| 빠른 시작/일반적인 경우 | XGBoost 또는 LightGBM |

### 💡 실무 팁

1. **기준 모델부터**: Linear Regression으로 시작하여 baseline 설정
2. **스케일링 주의**: 선형 모델은 스케일링 필수, 트리 모델은 불필요
3. **과적합 방지**: 정규화(Ridge/Lasso) 또는 트리 깊이 제한
4. **교차 검증**: GridSearchCV/RandomizedSearchCV 필수
5. **앙상블 효과**: 단일 모델보다 Random Forest, Boosting이 대체로 우수
6. **CatBoost 활용**: 범주형 특성 많으면 인코딩 없이 바로 사용